In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod 
import random
from numba import njit
from numba.experimental import jitclass
from numba import int32, float64
from sklearn import datasets
import mpl_toolkits.mplot3d 
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from cxplain.xkm import XkmExplainer
from cxplain.tree import  DecisionTreeExplainer, RandomForestExplainer, ExKMCExplainer
from cxplain.shap import  ShapExplainer
from cxplain.gradient import GradientExplainer  
from cxplain.metrics import EuclideanMetric, Metric, ManhattenMetric
from cxplain.neon import NeonKMeansExplainer
from cxplain.errors import NonExistingRelevanceError
from imputer import NormalCKDEImputer, EmpiricalRandomImputer, get_imputer
from datasets import IrisDataset, WineDataset, WholeSaleDataset, LiveSellersDataset, BuddyMoveDataset, SyntheticDataset

C:\Users\fottneal\Documents\code\cluster-explain\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Data sets to be considered:
- https://archive-beta.ics.uci.edu/dataset/292/wholesale+customers
- https://archive.ics.uci.edu/ml/datasets/BuddyMove+Data+Set# --> keine targets --> raus
- https://archive.ics.uci.edu/ml/datasets/Facebook+Live+Sellers+in+Thailand#
- https://archive.ics.uci.edu/ml/datasets/wine / https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html#sklearn.datasets.load_wine, included in scikit learn
- https://www.researchgate.net/publication/331616284_A_morphological_database_for_Colombian_anuran_species_from_conservation-priority_ecosystems
- https://archive.ics.uci.edu/ml/datasets/clickstream+data+for+online+shopping#
- https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html

In [3]:
datasets = {"iris": IrisDataset.load_and_clean_dataset(),
            "wine": WineDataset.load_and_clean_dataset(),
            "wholesale": WholeSaleDataset.load_and_clean_dataset("../data/Wholesale customers data.csv"),
            "buddy": BuddyMoveDataset.load_and_clean_dataset(3, "../data/buddymove_holidayiq.csv"),
            "synthetic": SyntheticDataset.load_and_clean_dataset(15, "../data/data_s1.txt"),
            "live_sellers": LiveSellersDataset.load_and_clean_dataset("../data/Live_20210128.csv")
           }

In [4]:
only_global = False
use_imputer = True
imputer_name = "random"
n_trials = 100

#np.random.seed(3)

dataset_results_local = {}

for dataset_name, dataset in datasets.items():
    print(dataset_name)

    n_clusters = dataset.n_clusters
    X = dataset.features
    y = dataset.targets
    n_obs = dataset.n_obs
    n_features = dataset.n_features

    # fit Kmeans
    kmeans = KMeans(n_clusters=n_clusters, random_state=3).fit(X)
    cluster_centers = kmeans.cluster_centers_
    predictions = kmeans.predict(X)
    # init and fit explainer
    # list allexplainers
    explainers = {"tree": DecisionTreeExplainer(data= X, cluster_predictions=predictions),
                 "forest": RandomForestExplainer(data= X, cluster_predictions=predictions),
                 "exkmc": ExKMCExplainer(X, kmeans, k=n_clusters, max_leaves=2*n_clusters),
                 "gradient": GradientExplainer(X, cluster_centers, predictions, EuclideanMetric, enable_abs_calculation=False),
                 "shap": ShapExplainer(data= X, cluster_predictions=predictions),
                 "neon": NeonKMeansExplainer(cluster_centers=cluster_centers, data=X, predictions=predictions),
                 "xkm_next_best": XkmExplainer(X,  kmeans.cluster_centers_, "next_best", "euclidean", predictions),
                 "xkm_all": XkmExplainer(X,  kmeans.cluster_centers_, "all", "euclidean", predictions)}

    # fit and explain all explainers

    explanations = {explainer_name:explainer.fit_explain() for explainer_name, explainer  in explainers.items()}

    # first calculate all ROC curves for individual observations
    result_individual = {explainer_name: [] for explainer_name in explanations.keys()}
    if use_imputer:
        imputer = get_imputer(imputer_name)(X).fit()
    for _ in range(n_trials):
        print(f"trial: {_ + 1}")
        for explainer_name, explanation in explanations.items():
            # init curve_list
            curve_list = []
            for index_obs in range(n_obs):
                # init list curve_obs_i to all 1 (length = num_features)
                curve_obs = [1 for i in range(n_features)]
                # init array of feature observations, I use an array instead of a list, as it is easier  later on to calculate distances to cluster centers
                feature_obs = np.array([0.0 for i in range(n_features)])
                # get relevance scores for observation, for explainers with only global scores, these will be used for every observation
                if only_global:
                    relevance_scores = list(explanations[explainer_name].global_relevance)
                else:
                    try:
                        relevance_scores = list(explanations[explainer_name].pointwise_relevance.iloc[index_obs, :])
                    except NonExistingRelevanceError:
                        relevance_scores = list(explanations[explainer_name].global_relevance)

                for feature_index in range(n_features):
                    # get biggest score and column index (indicate which feature is meant) and pop from list
                    index_biggest_score = relevance_scores.index(max(relevance_scores))                
                    relevance_scores[index_biggest_score] = -1000 # I set to large negative number as popping would ruin the index correspondence from relevance score to feature
                    # get observation for this feature
                    obs_biggest_score = X[index_obs, index_biggest_score]
                    # get corresponding cluster index for this observation
                    cluster_index = predictions[index_obs]
                    # add observation for feature to feature observations list
                    feature_obs.put(index_biggest_score, obs_biggest_score) # has to be at index of feature in training data, as otherwise distance calculation is wrong
                    # impute other entries (length = num_features) --> TBD
                    if use_imputer: 
                        if feature_index < (n_features - 1):
                            feature_obs_imputed = imputer.predict(feature_obs, index_obs)
                        else:
                            feature_obs_imputed = feature_obs.copy()
                    else:
                        feature_obs_imputed = feature_obs.copy()
                    # calculate distance to cluster centers for feature observations list
                    distances = [np.linalg.norm(feature_obs_imputed - center) for center in cluster_centers]
                    # get nearest_cluster_index
                    nearest_cluster_index = distances.index(min(distances))
                    # check whether cluster_index == nearest_cluster_index
                    # if yes: return curve_obs_i
                    # if no: replace first entry of curve_obs_i ith 0 and repeat
                    if cluster_index == nearest_cluster_index:
                        break
                    else:
                        curve_obs[feature_index] = 0
                    # if yes: return curve_obs_i
                    # if no: replace first entry of curve_obs_i ith 0 and repeat

                curve_list.append(curve_obs)

            # add explainer entry to dict
            result_individual[explainer_name].extend(curve_list)

    # Now compute AUC
    result_auc = {explainer_name: (1 /(n_obs*n_features*n_trials)) * sum(map(sum, curves)) for explainer_name, curves in result_individual.items()}

    dataset_results_local[dataset_name] = result_auc

iris
trial: 1
trial: 2
trial: 3
trial: 4
trial: 5
trial: 6
trial: 7
trial: 8
trial: 9
trial: 10
trial: 11
trial: 12
trial: 13
trial: 14
trial: 15
trial: 16
trial: 17
trial: 18
trial: 19
trial: 20
trial: 21
trial: 22
trial: 23
trial: 24
trial: 25
trial: 26
trial: 27
trial: 28
trial: 29
trial: 30
trial: 31
trial: 32
trial: 33
trial: 34
trial: 35
trial: 36
trial: 37
trial: 38
trial: 39
trial: 40
trial: 41
trial: 42
trial: 43
trial: 44
trial: 45
trial: 46
trial: 47
trial: 48
trial: 49
trial: 50
trial: 51
trial: 52
trial: 53
trial: 54
trial: 55
trial: 56
trial: 57
trial: 58
trial: 59
trial: 60
trial: 61
trial: 62
trial: 63
trial: 64
trial: 65
trial: 66
trial: 67
trial: 68
trial: 69
trial: 70
trial: 71
trial: 72
trial: 73
trial: 74
trial: 75
trial: 76
trial: 77
trial: 78
trial: 79
trial: 80
trial: 81
trial: 82
trial: 83
trial: 84
trial: 85
trial: 86
trial: 87
trial: 88
trial: 89
trial: 90
trial: 91
trial: 92
trial: 93
trial: 94
trial: 95
trial: 96
trial: 97
trial: 98
trial: 99
trial: 100
win

In [5]:
only_global = True
use_imputer = True
imputer_name = "random"
n_trials = 100

#np.random.seed(3)
dataset_results_global = {}

for dataset_name, dataset in datasets.items():
    print(dataset_name)

    n_clusters = dataset.n_clusters
    X = dataset.features
    y = dataset.targets
    n_obs = dataset.n_obs
    n_features = dataset.n_features

    # fit Kmeans
    kmeans = KMeans(n_clusters=n_clusters, random_state=3).fit(X)
    cluster_centers = kmeans.cluster_centers_
    predictions = kmeans.predict(X)
    # init and fit explainer
    # list allexplainers
    explainers = {"tree": DecisionTreeExplainer(data= X, cluster_predictions=predictions),
                 "forest": RandomForestExplainer(data= X, cluster_predictions=predictions),
                 "exkmc": ExKMCExplainer(X, kmeans, k=n_clusters, max_leaves=2*n_clusters),
                 "gradient": GradientExplainer(X, cluster_centers, predictions, EuclideanMetric, enable_abs_calculation=False),
                 "shap": ShapExplainer(data= X, cluster_predictions=predictions),
                 "neon": NeonKMeansExplainer(cluster_centers=cluster_centers, data=X, predictions=predictions),
                 "xkm_next_best": XkmExplainer(X,  kmeans.cluster_centers_, "next_best", "euclidean", predictions),
                 "xkm_all": XkmExplainer(X,  kmeans.cluster_centers_, "all", "euclidean", predictions)}

    # fit and explain all explainers

    explanations = {explainer_name:explainer.fit_explain() for explainer_name, explainer  in explainers.items()}

    # first calculate all ROC curves for individual observations
    result_individual = {explainer_name: [] for explainer_name in explanations.keys()}
    if use_imputer:
        imputer = get_imputer(imputer_name)(X).fit()
    for _ in range(n_trials):
        print(f"trial: {_ + 1}")
        for explainer_name, explanation in explanations.items():
            # init curve_list
            curve_list = []
            for index_obs in range(n_obs):
                # init list curve_obs_i to all 1 (length = num_features)
                curve_obs = [1 for i in range(n_features)]
                # init array of feature observations, I use an array instead of a list, as it is easier  later on to calculate distances to cluster centers
                feature_obs = np.array([0.0 for i in range(n_features)])
                # get relevance scores for observation, for explainers with only global scores, these will be used for every observation
                if only_global:
                    relevance_scores = list(explanations[explainer_name].global_relevance)
                else:
                    try:
                        relevance_scores = list(explanations[explainer_name].pointwise_relevance.iloc[index_obs, :])
                    except NonExistingRelevanceError:
                        relevance_scores = list(explanations[explainer_name].global_relevance)

                for feature_index in range(n_features):
                    # get biggest score and column index (indicate which feature is meant) and pop from list
                    index_biggest_score = relevance_scores.index(max(relevance_scores))                
                    relevance_scores[index_biggest_score] = -100 # I set to large negative number as popping would ruin the index correspondence from relevance score to feature
                    # get observation for this feature
                    obs_biggest_score = X[index_obs, index_biggest_score]
                    # get corresponding cluster index for this observation
                    cluster_index = predictions[index_obs]
                    # add observation for feature to feature observations list
                    feature_obs.put(index_biggest_score, obs_biggest_score) # has to be at index of feature in training data, as otherwise distance calculation is wrong
                    # impute other entries (length = num_features) --> TBD
                    if use_imputer: 
                        if feature_index < (n_features - 1):
                            feature_obs_imputed = imputer.predict(feature_obs, index_obs)
                        else:
                            feature_obs_imputed = feature_obs.copy()
                    else:
                        feature_obs_imputed = feature_obs.copy()
                    # calculate distance to cluster centers for feature observations list
                    distances = [np.linalg.norm(feature_obs_imputed - center) for center in cluster_centers]
                    # get nearest_cluster_index
                    nearest_cluster_index = distances.index(min(distances))
                    # check whether cluster_index == nearest_cluster_index
                    # if yes: return curve_obs_i
                    # if no: replace first entry of curve_obs_i ith 0 and repeat
                    if cluster_index == nearest_cluster_index:
                        break
                    else:
                        curve_obs[feature_index] = 0
                    # if yes: return curve_obs_i
                    # if no: replace first entry of curve_obs_i ith 0 and repeat

                curve_list.append(curve_obs)

            # add explainer entry to dict
            result_individual[explainer_name].extend(curve_list)

    # Now compute AUC
    result_auc = {explainer_name: (1 /(n_obs*n_features*n_trials)) * sum(map(sum, curves)) for explainer_name, curves in result_individual.items()}

    dataset_results_global[dataset_name] = result_auc

iris
trial: 1
trial: 2
trial: 3
trial: 4
trial: 5
trial: 6
trial: 7
trial: 8
trial: 9
trial: 10
trial: 11
trial: 12
trial: 13
trial: 14
trial: 15
trial: 16
trial: 17
trial: 18
trial: 19
trial: 20
trial: 21
trial: 22
trial: 23
trial: 24
trial: 25
trial: 26
trial: 27
trial: 28
trial: 29
trial: 30
trial: 31
trial: 32
trial: 33
trial: 34
trial: 35
trial: 36
trial: 37
trial: 38
trial: 39
trial: 40
trial: 41
trial: 42
trial: 43
trial: 44
trial: 45
trial: 46
trial: 47
trial: 48
trial: 49
trial: 50
trial: 51
trial: 52
trial: 53
trial: 54
trial: 55
trial: 56
trial: 57
trial: 58
trial: 59
trial: 60
trial: 61
trial: 62
trial: 63
trial: 64
trial: 65
trial: 66
trial: 67
trial: 68
trial: 69
trial: 70
trial: 71
trial: 72
trial: 73
trial: 74
trial: 75
trial: 76
trial: 77
trial: 78
trial: 79
trial: 80
trial: 81
trial: 82
trial: 83
trial: 84
trial: 85
trial: 86
trial: 87
trial: 88
trial: 89
trial: 90
trial: 91
trial: 92
trial: 93
trial: 94
trial: 95
trial: 96
trial: 97
trial: 98
trial: 99
trial: 100
win

In [6]:
dataset_results_local

{'iris': {'tree': 0.8236,
  'forest': 0.9478500000000001,
  'exkmc': 0.83345,
  'gradient': 0.8055833333333333,
  'shap': 0.9500166666666667,
  'neon': 0.7864333333333333,
  'xkm_next_best': 0.8693000000000001,
  'xkm_all': 0.8738333333333334},
 'wine': {'tree': 0.9340665514261021,
  'forest': 0.9344252376836647,
  'exkmc': 0.8932238547968886,
  'gradient': 0.9037294727744166,
  'shap': 0.9542480553154711,
  'neon': 0.8846629213483147,
  'xkm_next_best': 0.9262057044079517,
  'xkm_all': 0.9194770959377702},
 'wholesale': {'tree': 0.9453825757575758,
  'forest': 0.949094696969697,
  'exkmc': 0.9142954545454546,
  'gradient': 0.8977537878787879,
  'shap': 0.9658598484848485,
  'neon': 0.9404431818181818,
  'xkm_next_best': 0.9328371212121211,
  'xkm_all': 0.9008863636363637},
 'buddy': {'tree': 0.9016666666666666,
  'forest': 0.9000401606425702,
  'exkmc': 0.8949732262382865,
  'gradient': 0.834665327978581,
  'shap': 0.9195381526104417,
  'neon': 0.8018942436412315,
  'xkm_next_best': 0

In [7]:
dataset_results_global

{'iris': {'tree': 0.9509000000000001,
  'forest': 0.9476333333333333,
  'exkmc': 0.83525,
  'gradient': 0.8339166666666668,
  'shap': 0.9492166666666667,
  'neon': 0.7506333333333334,
  'xkm_next_best': 0.9458833333333334,
  'xkm_all': 0.9469833333333334},
 'wine': {'tree': 0.9325410544511669,
  'forest': 0.9341961970613657,
  'exkmc': 0.8928781331028522,
  'gradient': 0.8707346585998272,
  'shap': 0.9341356957649093,
  'neon': 0.9075064822817632,
  'xkm_next_best': 0.9160717372515126,
  'xkm_all': 0.9243388072601556},
 'wholesale': {'tree': 0.9486931818181817,
  'forest': 0.9515227272727272,
  'exkmc': 0.9152045454545454,
  'gradient': 0.9032992424242424,
  'shap': 0.9489431818181818,
  'neon': 0.8920151515151515,
  'xkm_next_best': 0.8744015151515151,
  'xkm_all': 0.9153219696969697},
 'buddy': {'tree': 0.893768406961178,
  'forest': 0.9020214190093708,
  'exkmc': 0.895863453815261,
  'gradient': 0.837764390896921,
  'shap': 0.900475234270415,
  'neon': 0.8023360107095047,
  'xkm_nex

In [9]:
# Serializing json
import json
json_local_results = json.dumps(dataset_results_local, indent=4)
 
# Writing to sample.json
with open("results_local.json", "w") as outfile:
    outfile.write(json_local_results)

In [10]:
# Serializing json
json_global_results = json.dumps(dataset_results_global, indent=4)
 
# Writing to sample.json
with open("results_global.json", "w") as outfile:
    outfile.write(json_global_results)